In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.models.forecasting.linear_regression_model import LinearRegressionModel
from sktime.datasets import load_airline
from sktime.forecasting.darts import DartsLinearRegressionModel
from sktime.forecasting.model_selection import temporal_train_test_split

In [ ]:
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

In [ ]:
y = load_airline()
y.head()

In [ ]:
y = pd.read_parquet("../data/energy.parquet")
y = TimeSeries.from_dataframe(y).drop_before(pd.Timestamp("2022"))
len(y)

In [ ]:
model = LinearRegressionModel(
    lags=[-24, -24 * 7],
    likelihood="quantile",
    quantiles=quantiles,
    output_chunk_length=5,
    multi_models=False,
    solver="highs",
)
model.fit(y)

In [ ]:
model.predict(n=5, predict_likelihood_parameters=True)

## SKTIME

In [ ]:
y_df = y.pd_dataframe()
y_df.head()

In [ ]:
forecaster = DartsLinearRegressionModel(
    lags=[-24, -24 * 7],
    likelihood="quantile",
    quantiles=quantiles,
    output_chunk_length=5,
    multi_models=False,
)
forecaster.fit(y_df)
y_pred = forecaster.predict_quantiles(fh=[1, 2, 3, 4, 5], alpha=quantiles)
y_pred

In [ ]:
# bikes = pd.read_parquet("../data/bikes.parquet")
bikes = pd.read_parquet("../data/energy.parquet")
bikes.head()

In [ ]:
forecaster = DartsLinearRegressionModel(
    lags=[-7, -14],
    likelihood="quantile",
    quantiles=quantiles,
    output_chunk_length=1,
    multi_models=False,
)
forecaster.fit(bikes)
y_pred = forecaster.predict_quantiles(fh=[1, 2, 3, 4, 5], alpha=quantiles)
y_pred

In [ ]:
# bikes = load_airline()
bikes = pd.read_parquet("../data/bikes.parquet")

In [ ]:
train, test = temporal_train_test_split(bikes, test_size=0.2)
len(train), len(test)

In [ ]:
import numpy as np

forecaster = DartsLinearRegressionModel(
    lags=3,
    likelihood="quantile",
    quantiles=quantiles,
    output_chunk_length=1,
    multi_models=False,
)
forecaster.fit(train)
fh = np.arange(1, len(test) + 1)
y_pred = forecaster.predict_quantiles(fh=fh, alpha=quantiles)

In [ ]:
y_pred